# 九个站点批量计算

In [1]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import tensorflow as tf
import keras
import catboost as cb
from pandas import read_csv
from math import sqrt
from numpy import concatenate
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn import svm
from keras.layers.core import Dense, Activation, Dropout
from sklearn.externals import joblib
%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:472: FutureWarning: Passi

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto(allow_soft_placement = True)
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.5)
config.gpu_options.allow_growth = True
sess0 = tf.InteractiveSession(config = config)

In [3]:
print(os.getcwd()) # 打印当前工作目录

E:\Research Project\DKASC, Alice Springs


In [4]:
os.listdir(os.getcwd())

['.idea',
 '.ipynb_checkpoints',
 '101-Site_DKA-WeatherStation',
 '96-Site_DKA-MasterMeter1',
 'Alice_Springs_2016-2020',
 'Dataset',
 'logs',
 'picture',
 'Predict',
 'Program backups',
 'temp',
 'venv',
 '数据集_小波包分解重构数据.ipynb',
 '数据集_预处理_第二版0702.ipynb',
 '测试小波分解.ipynb',
 '测试模型0622.ipynb',
 '测试模型0704.ipynb',
 '测试模型0704批量.ipynb',
 '画图.ipynb',
 '画图0704.ipynb',
 '笔记.ipynb',
 '训练模型0622.ipynb',
 '训练模型0702.ipynb',
 '训练模型0703 WPD.ipynb']

In [5]:
os.chdir("./Dataset/test0702/")

In [6]:
os.listdir()

['test_Station_1.csv',
 'test_Station_2.csv',
 'test_Station_3.csv',
 'test_Station_4.csv',
 'test_Station_5.csv',
 'test_Station_6.csv',
 'test_Station_7.csv',
 'test_Station_8.csv',
 'test_Station_9.csv']

In [7]:
file_chdir = os.getcwd()

In [8]:
file_chdir

'E:\\Research Project\\DKASC, Alice Springs\\Dataset\\test0702'

In [9]:
filecsv_list = []
for root,dirs,files in os.walk(file_chdir):
    for file in files:
        if os.path.splitext(file)[1] == '.csv':
            filecsv_list.append(file)

In [10]:
filecsv_list

['test_Station_1.csv',
 'test_Station_2.csv',
 'test_Station_3.csv',
 'test_Station_4.csv',
 'test_Station_5.csv',
 'test_Station_6.csv',
 'test_Station_7.csv',
 'test_Station_8.csv',
 'test_Station_9.csv']

In [11]:
testnames=list()
testnames += [('test_%d' % (j)) for j in range(len(filecsv_list))]

In [12]:
testnames

['test_0',
 'test_1',
 'test_2',
 'test_3',
 'test_4',
 'test_5',
 'test_6',
 'test_7',
 'test_8']

In [13]:
for csv in filecsv_list:
    testnames[filecsv_list.index(csv)]=pd.read_csv(csv)

In [14]:
testnames[0]

,Timestamp,Year,Month,Day,Hour,Minutes,Temperature(°C),Relative Humidity(%),Wind Direction(Degrees),Daily Rainfall(mm),Global Horizontal Radiation(W/m²),Diffuse Horizontal Radiation(W/m²),Station_1
0,2019/1/1 0:00,2019,1,1,0,0,30.020723,20.125513,32.213470,0.0,1.543787,1.349848,0.0
1,2019/1/1 0:05,2019,1,1,0,5,29.958653,20.040049,32.158791,0.0,1.373370,1.244123,0.0
2,2019/1/1 0:10,2019,1,1,0,10,29.884491,20.013313,32.155808,0.0,1.142953,0.992849,0.0
3,2019/1/1 0:15,2019,1,1,0,15,29.831554,20.068233,32.178555,0.0,0.709036,0.611533,0.0
4,2019/1/1 0:20,2019,1,1,0,20,29.750330,20.086567,32.166214,0.0,1.058623,0.814599,0.0
5,2019/1/1 0:25,2019,1,1,0,25,29.532784,20.429800,32.247841,0.0,1.458448,1.263590,0.0
6,2019/1/1 0:30,2019,1,1,0,30,29.333590,20.735050,32.305141,0.0,1.684532,1.487528,0.0
7,2019/1/1 0:35,2019,1,1,0,35,29.115730,20.767771,32.390381,0.0,1.785272,1.607798,0.0
8,2019/1/1 0:40,2019,1,1,0,40,28.895197,21.143364,32.458775,0.0,1.507639,1.279625,0.0
9,2019/1/1 0:45,2019,1,1,0,45,28.870382,21.860403,32.498596,0.0,1.136652,0.865316,0.0


In [15]:
#查看输入数据维度
# print(train.shape)
for tena in testnames:
    print(tena.shape)

(102888, 13)
(102888, 13)
(102888, 13)
(102888, 13)
(102888, 13)
(102888, 13)
(102888, 13)
(102888, 13)
(102888, 13)


In [16]:
for tena in testnames:
    print(tena.head(1))
# test.head(5)

       Timestamp  Year  Month  Day  Hour  Minutes  Temperature(°C)  \
0  2019/1/1 0:00  2019      1    1     0        0        30.020723   

   Relative Humidity(%)  Wind Direction(Degrees)  Daily Rainfall(mm)  \
0             20.125513                 32.21347                 0.0   

   Global Horizontal Radiation(W/m²)  Diffuse Horizontal Radiation(W/m²)  \
0                           1.543787                            1.349848   

   Station_1  
0        0.0  
       Timestamp  Year  Month  Day  Hour  Minutes  Temperature(°C)  \
0  2019/1/1 0:00  2019      1    1     0        0        30.020723   

   Relative Humidity(%)  Wind Direction(Degrees)  Daily Rainfall(mm)  \
0             20.125513                 32.21347                 0.0   

   Global Horizontal Radiation(W/m²)  Diffuse Horizontal Radiation(W/m²)  \
0                           1.543787                            1.349848   

   Station_2  
0        0.0  
       Timestamp  Year  Month  Day  Hour  Minutes  Temperature

# 数据预处理训练集去异常值

In [17]:
# test.isnull().sum()
for tena in testnames:
    print(tena.isnull().sum())

Timestamp                             0
Year                                  0
Month                                 0
Day                                   0
Hour                                  0
Minutes                               0
Temperature(°C)                       0
Relative Humidity(%)                  0
Wind Direction(Degrees)               0
Daily Rainfall(mm)                    0
Global Horizontal Radiation(W/m²)     0
Diffuse Horizontal Radiation(W/m²)    0
Station_1                             0
dtype: int64
Timestamp                             0
Year                                  0
Month                                 0
Day                                   0
Hour                                  0
Minutes                               0
Temperature(°C)                       0
Relative Humidity(%)                  0
Wind Direction(Degrees)               0
Daily Rainfall(mm)                    0
Global Horizontal Radiation(W/m²)     0
Diffuse Horizontal Radiatio

In [18]:
# test.columns
for tena in testnames:
    print(tena.columns)

Index(['Timestamp', 'Year', 'Month', 'Day', 'Hour', 'Minutes',
       'Temperature(°C)', 'Relative Humidity(%)', 'Wind Direction(Degrees)',
       'Daily Rainfall(mm)', 'Global Horizontal Radiation(W/m²)',
       'Diffuse Horizontal Radiation(W/m²)', 'Station_1'],
      dtype='object')
Index(['Timestamp', 'Year', 'Month', 'Day', 'Hour', 'Minutes',
       'Temperature(°C)', 'Relative Humidity(%)', 'Wind Direction(Degrees)',
       'Daily Rainfall(mm)', 'Global Horizontal Radiation(W/m²)',
       'Diffuse Horizontal Radiation(W/m²)', 'Station_2'],
      dtype='object')
Index(['Timestamp', 'Year', 'Month', 'Day', 'Hour', 'Minutes',
       'Temperature(°C)', 'Relative Humidity(%)', 'Wind Direction(Degrees)',
       'Daily Rainfall(mm)', 'Global Horizontal Radiation(W/m²)',
       'Diffuse Horizontal Radiation(W/m²)', 'Station_3'],
      dtype='object')
Index(['Timestamp', 'Year', 'Month', 'Day', 'Hour', 'Minutes',
       'Temperature(°C)', 'Relative Humidity(%)', 'Wind Direction(Degrees)',

In [19]:
len(testnames)

9

In [20]:
for tena in range(0,len(testnames)):
    testnames[tena] = testnames[tena].drop(columns=['Timestamp','Year'])
    print(testnames[tena].columns)

Index(['Month', 'Day', 'Hour', 'Minutes', 'Temperature(°C)',
       'Relative Humidity(%)', 'Wind Direction(Degrees)', 'Daily Rainfall(mm)',
       'Global Horizontal Radiation(W/m²)',
       'Diffuse Horizontal Radiation(W/m²)', 'Station_1'],
      dtype='object')
Index(['Month', 'Day', 'Hour', 'Minutes', 'Temperature(°C)',
       'Relative Humidity(%)', 'Wind Direction(Degrees)', 'Daily Rainfall(mm)',
       'Global Horizontal Radiation(W/m²)',
       'Diffuse Horizontal Radiation(W/m²)', 'Station_2'],
      dtype='object')
Index(['Month', 'Day', 'Hour', 'Minutes', 'Temperature(°C)',
       'Relative Humidity(%)', 'Wind Direction(Degrees)', 'Daily Rainfall(mm)',
       'Global Horizontal Radiation(W/m²)',
       'Diffuse Horizontal Radiation(W/m²)', 'Station_3'],
      dtype='object')
Index(['Month', 'Day', 'Hour', 'Minutes', 'Temperature(°C)',
       'Relative Humidity(%)', 'Wind Direction(Degrees)', 'Daily Rainfall(mm)',
       'Global Horizontal Radiation(W/m²)',
       'Diffuse H

In [22]:
testnames[0].head()

,Month,Day,Hour,Minutes,Temperature(°C),Relative Humidity(%),Wind Direction(Degrees),Daily Rainfall(mm),Global Horizontal Radiation(W/m²),Diffuse Horizontal Radiation(W/m²),Station_1
0,1,1,0,0,30.020723,20.125513,32.213470,0.0,1.543787,1.349848,0.0
1,1,1,0,5,29.958653,20.040049,32.158791,0.0,1.373370,1.244123,0.0
2,1,1,0,10,29.884491,20.013313,32.155808,0.0,1.142953,0.992849,0.0
3,1,1,0,15,29.831554,20.068233,32.178555,0.0,0.709036,0.611533,0.0
4,1,1,0,20,29.750330,20.086567,32.166214,0.0,1.058623,0.814599,0.0


In [23]:
testnames[0].head()

,Month,Day,Hour,Minutes,Temperature(°C),Relative Humidity(%),Wind Direction(Degrees),Daily Rainfall(mm),Global Horizontal Radiation(W/m²),Diffuse Horizontal Radiation(W/m²),Station_1
0,1,1,0,0,30.020723,20.125513,32.213470,0.0,1.543787,1.349848,0.0
1,1,1,0,5,29.958653,20.040049,32.158791,0.0,1.373370,1.244123,0.0
2,1,1,0,10,29.884491,20.013313,32.155808,0.0,1.142953,0.992849,0.0
3,1,1,0,15,29.831554,20.068233,32.178555,0.0,0.709036,0.611533,0.0
4,1,1,0,20,29.750330,20.086567,32.166214,0.0,1.058623,0.814599,0.0


# keras模型训练

准备数据

In [24]:
for tena in testnames:
    print(tena.head())

   Month  Day  Hour  Minutes  Temperature(°C)  Relative Humidity(%)  \
0      1    1     0        0        30.020723             20.125513   
1      1    1     0        5        29.958653             20.040049   
2      1    1     0       10        29.884491             20.013313   
3      1    1     0       15        29.831554             20.068233   
4      1    1     0       20        29.750330             20.086567   

   Wind Direction(Degrees)  Daily Rainfall(mm)  \
0                32.213470                 0.0   
1                32.158791                 0.0   
2                32.155808                 0.0   
3                32.178555                 0.0   
4                32.166214                 0.0   

   Global Horizontal Radiation(W/m²)  Diffuse Horizontal Radiation(W/m²)  \
0                           1.543787                            1.349848   
1                           1.373370                            1.244123   
2                           1.142953        

# 构建序列

In [25]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
#     n_vars = 1 if type(data) is list else data.shape[1]
    n_vars = list(data)
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in n_vars]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('%s(t)' % (j) )for j in n_vars]
        else:
            names += [('%s(t+%d)' % (j, i)) for j in n_vars]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [26]:
# test=glocals()
# test += [('test%d' % (j)) for j in range(len(testnames))]

In [27]:
for i in range(0, 9):
    exec("test%d=series_to_supervised(testnames[i],12,1)"%i)


In [28]:
test0.columns

Index(['Month(t-12)', 'Day(t-12)', 'Hour(t-12)', 'Minutes(t-12)',
       'Temperature(°C)(t-12)', 'Relative Humidity(%)(t-12)',
       'Wind Direction(Degrees)(t-12)', 'Daily Rainfall(mm)(t-12)',
       'Global Horizontal Radiation(W/m²)(t-12)',
       'Diffuse Horizontal Radiation(W/m²)(t-12)',
       ...
       'Day(t)', 'Hour(t)', 'Minutes(t)', 'Temperature(°C)(t)',
       'Relative Humidity(%)(t)', 'Wind Direction(Degrees)(t)',
       'Daily Rainfall(mm)(t)', 'Global Horizontal Radiation(W/m²)(t)',
       'Diffuse Horizontal Radiation(W/m²)(t)', 'Station_1(t)'],
      dtype='object', length=143)

In [29]:
# for i in range(0, 9):
#     exec("test%d=test%d.drop(test%d.columns[[0,14,15,16,17,18,19,20,21,22,23,24,25,26,28,29,30,31,32,33,34,35,36,37,38,39,40]],axis=1,inplace=True)"%i)

# 重新选择

In [30]:
test0.drop(test0.columns[11:142],axis=1,inplace=True)

In [31]:
test1.drop(test1.columns[11:142],axis=1,inplace=True)

In [32]:
test2.drop(test2.columns[11:142],axis=1,inplace=True)

In [33]:
test3.drop(test3.columns[11:142],axis=1,inplace=True)

In [34]:
test4.drop(test4.columns[11:142],axis=1,inplace=True)

In [35]:
test5.drop(test5.columns[11:142],axis=1,inplace=True)

In [36]:
test6.drop(test6.columns[11:142],axis=1,inplace=True)

In [37]:
test7.drop(test7.columns[11:142],axis=1,inplace=True)

In [38]:
test8.drop(test8.columns[11:142],axis=1,inplace=True)

In [39]:
print(test0.shape,test1.shape,test2.shape,test3.shape,test4.shape,test5.shape,test6.shape,test7.shape,test8.shape,)

(102876, 12) (102876, 12) (102876, 12) (102876, 12) (102876, 12) (102876, 12) (102876, 12) (102876, 12) (102876, 12)


In [40]:
test0.columns

Index(['Month(t-12)', 'Day(t-12)', 'Hour(t-12)', 'Minutes(t-12)',
       'Temperature(°C)(t-12)', 'Relative Humidity(%)(t-12)',
       'Wind Direction(Degrees)(t-12)', 'Daily Rainfall(mm)(t-12)',
       'Global Horizontal Radiation(W/m²)(t-12)',
       'Diffuse Horizontal Radiation(W/m²)(t-12)', 'Station_1(t-12)',
       'Station_1(t)'],
      dtype='object')

# 归一化数据

In [ ]:
# train1 = train1.values
# train1 = train1.astype('float32')     # 将整型变为float
# scaler = MinMaxScaler(feature_range=(0, 1)) # 数据处理，归一化至0~1之间
# train1 = scaler.fit_transform(train1)

# test1 = test1.values
# test1 = test1.astype('float32')     # 将整型变为float
# scaler = MinMaxScaler(feature_range=(0, 1)) # 数据处理，归一化至0~1之间
# test1 = scaler.fit_transform(test1)

# inds = np.arange(train1.shape[0])
# train1_inds = inds[0:int(train1.shape[0]*0.8)]
# eval1_inds = inds[int(train1.shape[0]*0.8):train1.shape[0]]


# train_X, train_y = train1[train1_inds,:-1], train1[train1_inds,-1:]
# eval_X,eval_y = train1[eval1_inds,:-1], train1[eval1_inds,-1:]
# test_X, test_y = test1[:, :-1], test1[:,-1:]

# # reshape input to be 3D [samples, timesteps, features]
# train_Xl = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
# eval_Xl = eval_X.reshape((eval_X.shape[0],1,eval_X.shape[1]))
# test_Xl = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
# print(train_X.shape, train_y.shape,eval_X.shape,eval_y.shape, test_X.shape, test_y.shape)
# print(train_Xl.shape, train_y.shape,eval_Xl.shape,eval_y.shape, test_Xl.shape, test_y.shape)

0

In [41]:
test0 = test0.values
test0 = test0.astype('float32')     # 将整型变为float
scaler = MinMaxScaler(feature_range=(0, 1)) # 数据处理，归一化至0~1之间
test0 = scaler.fit_transform(test0)

test0_X, test0_y = test0[:, :-1], test0[:,-1:]

# reshape input to be 3D [samples, timesteps, features]
test0_Xl = test0_X.reshape((test0_X.shape[0], 1, test0_X.shape[1]))
print(test0_X.shape, test0_y.shape)
print(test0_Xl.shape, test0_y.shape)

(102876, 11) (102876, 1)
(102876, 1, 11) (102876, 1)


1

In [42]:
test1 = test1.values
test1 = test1.astype('float32')     # 将整型变为float
scaler = MinMaxScaler(feature_range=(0, 1)) # 数据处理，归一化至0~1之间
test1 = scaler.fit_transform(test1)

test1_X, test1_y = test1[:, :-1], test1[:,-1:]

# reshape input to be 3D [samples, timesteps, features]
test1_Xl = test1_X.reshape((test1_X.shape[0], 1, test1_X.shape[1]))
print(test1_X.shape, test1_y.shape)
print(test1_Xl.shape, test1_y.shape)

(102876, 11) (102876, 1)
(102876, 1, 11) (102876, 1)


2

In [43]:
test2 = test2.values
test2 = test2.astype('float32')     # 将整型变为float
scaler = MinMaxScaler(feature_range=(0, 1)) # 数据处理，归一化至0~1之间
test2 = scaler.fit_transform(test2)

test2_X, test2_y = test2[:, :-1], test2[:,-1:]

# reshape input to be 3D [samples, timesteps, features]
test2_Xl = test2_X.reshape((test2_X.shape[0], 1, test2_X.shape[1]))
print(test2_X.shape, test2_y.shape)
print(test2_Xl.shape, test2_y.shape)

(102876, 11) (102876, 1)
(102876, 1, 11) (102876, 1)


3

In [44]:
test3 = test3.values
test3 = test3.astype('float32')     # 将整型变为float
scaler = MinMaxScaler(feature_range=(0, 1)) # 数据处理，归一化至0~1之间
test3 = scaler.fit_transform(test3)

test3_X, test3_y = test3[:, :-1], test3[:,-1:]

# reshape input to be 3D [samples, timesteps, features]
test3_Xl = test3_X.reshape((test3_X.shape[0], 1, test3_X.shape[1]))
print(test3_X.shape, test3_y.shape)
print(test3_Xl.shape, test3_y.shape)

(102876, 11) (102876, 1)
(102876, 1, 11) (102876, 1)


4

In [45]:
test4 = test4.values
test4 = test4.astype('float32')     # 将整型变为float
scaler = MinMaxScaler(feature_range=(0, 1)) # 数据处理，归一化至0~1之间
test4 = scaler.fit_transform(test4)

test4_X, test4_y = test4[:, :-1], test4[:,-1:]

# reshape input to be 3D [samples, timesteps, features]
test4_Xl = test4_X.reshape((test4_X.shape[0], 1, test4_X.shape[1]))
print(test4_X.shape, test4_y.shape)
print(test4_Xl.shape, test4_y.shape)

(102876, 11) (102876, 1)
(102876, 1, 11) (102876, 1)


5

In [46]:
test5 = test5.values
test5 = test5.astype('float32')     # 将整型变为float
scaler = MinMaxScaler(feature_range=(0, 1)) # 数据处理，归一化至0~1之间
test5 = scaler.fit_transform(test5)

test5_X, test5_y = test5[:, :-1], test5[:,-1:]

# reshape input to be 3D [samples, timesteps, features]
test5_Xl = test5_X.reshape((test5_X.shape[0], 1, test5_X.shape[1]))
print(test5_X.shape, test5_y.shape)
print(test5_Xl.shape, test5_y.shape)

(102876, 11) (102876, 1)
(102876, 1, 11) (102876, 1)


6

In [47]:
test6 = test6.values
test6 = test6.astype('float32')     # 将整型变为float
scaler = MinMaxScaler(feature_range=(0, 1)) # 数据处理，归一化至0~1之间
test6 = scaler.fit_transform(test6)

test6_X, test6_y = test6[:, :-1], test6[:,-1:]

# reshape input to be 3D [samples, timesteps, features]
test6_Xl = test6_X.reshape((test6_X.shape[0], 1, test6_X.shape[1]))
print(test6_X.shape, test6_y.shape)
print(test6_Xl.shape, test6_y.shape)

(102876, 11) (102876, 1)
(102876, 1, 11) (102876, 1)


7

In [48]:
test7 = test7.values
test7 = test7.astype('float32')     # 将整型变为float
scaler = MinMaxScaler(feature_range=(0, 1)) # 数据处理，归一化至0~1之间
test7 = scaler.fit_transform(test7)

test7_X, test7_y = test7[:, :-1], test7[:,-1:]

# reshape input to be 3D [samples, timesteps, features]
test7_Xl = test7_X.reshape((test7_X.shape[0], 1, test7_X.shape[1]))
print(test7_X.shape, test7_y.shape)
print(test7_Xl.shape, test7_y.shape)

(102876, 11) (102876, 1)
(102876, 1, 11) (102876, 1)


8

In [49]:
test8 = test8.values
test8 = test8.astype('float32')     # 将整型变为float
scaler = MinMaxScaler(feature_range=(0, 1)) # 数据处理，归一化至0~1之间
test8 = scaler.fit_transform(test8)

test8_X, test8_y = test8[:, :-1], test8[:,-1:]

# reshape input to be 3D [samples, timesteps, features]
test8_Xl = test8_X.reshape((test8_X.shape[0], 1, test8_X.shape[1]))
print(test8_X.shape, test8_y.shape)
print(test8_Xl.shape, test8_y.shape)

(102876, 11) (102876, 1)
(102876, 1, 11) (102876, 1)


# SVM model

In [50]:
os.chdir("../")

In [51]:
print(os.getcwd()) # 打印当前工作目录

E:\Research Project\DKASC, Alice Springs\Dataset


In [52]:
os.listdir()

['DKA.Station_1-9.csv',
 'NaN_DKA.Station.csv',
 'raw_DKA.Station_1-9.csv',
 'test',
 'test0702',
 'testdata.csv',
 'testWPD',
 'test_1A.csv',
 'train',
 'train0702',
 'traindata.csv',
 'trainWPD',
 'train_1A.csv',
 '测试集8个站点.csv',
 '训练集8个站点.csv']

# BP model

In [57]:
print(os.getcwd()) # 打印当前工作目录

E:\Research Project\DKASC, Alice Springs\Dataset


In [60]:
#模型加载
from keras.models import load_model
modelbp = load_model('../logs/BP0702.h5') 

# DNN model

In [64]:
#模型加载
modeldnn = load_model('../logs/DNN0702.h5') 

# LSTM model

In [68]:
#模型加载
modellstm = load_model('../logs/LSTM0702.h5') 

# 结果对比

反归一化

In [69]:
def inv(testPredictML,test_X,test_y):
    # invert scaling for forecast
    inv_testPredictML = concatenate((test_X[:,:],testPredictML), axis=1)
    inv_testPredictML = scaler.inverse_transform(inv_testPredictML)
    testPredictML = inv_testPredictML[:,-1:]
    # invert scaling for actual
    test_y = test_y.reshape((len(test_y), 1))
    inv_test = concatenate((test_X[:,:],test_y), axis=1)
    inv_test = scaler.inverse_transform(inv_test)
    testReal = inv_test[:,-1:]
    return testPredictML,testReal,inv_test

In [70]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred - y_true) / y_true)) * 100

def smape(y_true, y_pred):
    return 2.0 * np.mean(np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true))) * 100

# 0

In [71]:
test_X,test_Xl,test_y= test0_X,test0_Xl,test0_y

bp

In [76]:
bp_pred = modelbp.predict(test_X)
bp_pred,y_true,inv_test=inv(bp_pred,test_X,test_y)

bp_mse = metrics.mean_squared_error(y_true, bp_pred)
bp_rmse = math.sqrt(mean_squared_error(y_true[:,0], bp_pred[:,0]))
bp_mae = metrics.mean_absolute_error(y_true, bp_pred)
bp_mape = mape(y_true, bp_pred)
bp_smape = smape(y_true, bp_pred)

In [77]:
print(bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape)

0.27908564 0.5282855630612254 0.26321155 400.7814407348633 79.80666756629944


dnn

In [78]:
dnn_pred = modeldnn.predict(test_X)
# dnn_pred,y_true,inv_test=inv(dnn_pred,test_year,test_X,test_y)
dnn_pred,y_true,inv_test=inv(dnn_pred,test_X,test_y)

dnn_mse = metrics.mean_squared_error(y_true, dnn_pred)
dnn_rmse = math.sqrt(mean_squared_error(y_true[:,0], dnn_pred[:,0]))
dnn_mae = metrics.mean_absolute_error(y_true, dnn_pred)
dnn_mape = mape(y_true, dnn_pred)
dnn_smape = smape(y_true, dnn_pred)

In [79]:
print(dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape)

0.24960808 0.49960793084121535 0.23988028 134.10544395446777 74.14579391479492


LSTM


In [80]:
# make a prediction
lstm_pred = modellstm.predict(test_Xl)
# test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
lstm_pred,y_true,inv_test=inv(lstm_pred,test_X,test_y)
lstm_mse = metrics.mean_squared_error(y_true, lstm_pred)
lstm_rmse = math.sqrt(mean_squared_error(y_true[:,0], lstm_pred[:,0]))
lstm_mae = metrics.mean_absolute_error(y_true, lstm_pred)
lstm_mape = mape(y_true, lstm_pred)
lstm_smape = smape(y_true, lstm_pred)


In [81]:
print(lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape)

0.29763654 0.5455607562352454 0.28134742 139.33480978012085 59.062641859054565


In [82]:
bp0_mse,bp0_rmse,bp0_mae,bp0_mape,bp0_smape = bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape

In [83]:
dnn0_mse,dnn0_rmse,dnn0_mae,dnn0_mape,dnn0_smape = dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape

In [84]:
lstm0_mse,lstm0_rmse,lstm0_mae,lstm0_mape,lstm0_smape = lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape

# 1

In [85]:
test_X,test_Xl,test_y= test1_X,test1_Xl,test1_y

bp

In [86]:
bp_pred = modelbp.predict(test_X)
bp_pred,y_true,inv_test=inv(bp_pred,test_X,test_y)

bp_mse = metrics.mean_squared_error(y_true, bp_pred)
bp_rmse = math.sqrt(mean_squared_error(y_true[:,0], bp_pred[:,0]))
bp_mae = metrics.mean_absolute_error(y_true, bp_pred)
bp_mape = mape(y_true, bp_pred)
bp_smape = smape(y_true, bp_pred)

In [87]:
print(bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape)

0.23526673 0.48504301890587337 0.26173678 195.05741596221924 70.66615223884583


dnn

In [88]:
dnn_pred = modeldnn.predict(test_X)
# dnn_pred,y_true,inv_test=inv(dnn_pred,test_year,test_X,test_y)
dnn_pred,y_true,inv_test=inv(dnn_pred,test_X,test_y)

dnn_mse = metrics.mean_squared_error(y_true, dnn_pred)
dnn_rmse = math.sqrt(mean_squared_error(y_true[:,0], dnn_pred[:,0]))
dnn_mae = metrics.mean_absolute_error(y_true, dnn_pred)
dnn_mape = mape(y_true, dnn_pred)
dnn_smape = smape(y_true, dnn_pred)

In [89]:
print(dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape)

0.2188498 0.46781384446862534 0.25022665 95.65991163253784 89.05004858970642


LSTM


In [90]:
# make a prediction
lstm_pred = modellstm.predict(test_Xl)
# test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
lstm_pred,y_true,inv_test=inv(lstm_pred,test_X,test_y)
lstm_mse = metrics.mean_squared_error(y_true, lstm_pred)
lstm_rmse = math.sqrt(mean_squared_error(y_true[:,0], lstm_pred[:,0]))
lstm_mae = metrics.mean_absolute_error(y_true, lstm_pred)
lstm_mape = mape(y_true, lstm_pred)
lstm_smape = smape(y_true, lstm_pred)


In [91]:
print(lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape)

0.24340823 0.49336419907338475 0.26057783 99.77616667747498 71.02371454238892


In [92]:
bp1_mse,bp1_rmse,bp1_mae,bp1_mape,bp1_smape = bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape

In [93]:
dnn1_mse,dnn1_rmse,dnn1_mae,dnn1_mape,dnn1_smape = dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape

In [94]:
lstm1_mse,lstm1_rmse,lstm1_mae,lstm1_mape,lstm1_smape = lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape

# 2

In [95]:
test_X,test_Xl,test_y= test2_X,test2_Xl,test2_y

In [96]:
# svm_mse = metrics.mean_squared_error(y_true, svm_pred)
# # svm_rmse = math.sqrt(mean_squared_error(y_true[:,0], svm_pred[:,0]))
# svm_rmse = math.sqrt(mean_squared_error(y_true, svm_pred))
# svm_mae = metrics.mean_absolute_error(y_true, svm_pred)
# svm_mape = mape(y_true, svm_pred)
# svm_smape = smape(y_true, svm_pred)

In [97]:
# print(svm_mse,svm_rmse,svm_mae,svm_mape,svm_smape)

bp

In [98]:
bp_pred = modelbp.predict(test_X)
bp_pred,y_true,inv_test=inv(bp_pred,test_X,test_y)

bp_mse = metrics.mean_squared_error(y_true, bp_pred)
bp_rmse = math.sqrt(mean_squared_error(y_true[:,0], bp_pred[:,0]))
bp_mae = metrics.mean_absolute_error(y_true, bp_pred)
bp_mape = mape(y_true, bp_pred)
bp_smape = smape(y_true, bp_pred)

In [99]:
print(bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape)

0.11718605 0.34232447639160074 0.14815888 764.2271518707275 55.739933252334595


dnn

In [100]:
dnn_pred = modeldnn.predict(test_X)
# dnn_pred,y_true,inv_test=inv(dnn_pred,test_year,test_X,test_y)
dnn_pred,y_true,inv_test=inv(dnn_pred,test_X,test_y)

dnn_mse = metrics.mean_squared_error(y_true, dnn_pred)
dnn_rmse = math.sqrt(mean_squared_error(y_true[:,0], dnn_pred[:,0]))
dnn_mae = metrics.mean_absolute_error(y_true, dnn_pred)
dnn_mape = mape(y_true, dnn_pred)
dnn_smape = smape(y_true, dnn_pred)

In [101]:
print(dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape)

0.10333169 0.3214524731850787 0.14059035 322.71130084991455 99.73500370979309


LSTM


In [102]:
# make a prediction
lstm_pred = modellstm.predict(test_Xl)
# test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
lstm_pred,y_true,inv_test=inv(lstm_pred,test_X,test_y)
lstm_mse = metrics.mean_squared_error(y_true, lstm_pred)
lstm_rmse = math.sqrt(mean_squared_error(y_true[:,0], lstm_pred[:,0]))
lstm_mae = metrics.mean_absolute_error(y_true, lstm_pred)
lstm_mape = mape(y_true, lstm_pred)
lstm_smape = smape(y_true, lstm_pred)


In [103]:
print(lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape)

0.11144693 0.333836684560278 0.15925887 706.6081523895264 91.61741137504578


In [104]:
# svm2_mse,svm2_rmse,svm2_mae,svm2_mape,svm2_smape = svm_mse,svm_rmse,svm_mae,svm_mape,svm_smape

In [105]:
bp2_mse,bp2_rmse,bp2_mae,bp2_mape,bp2_smape = bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape

In [106]:
dnn2_mse,dnn2_rmse,dnn2_mae,dnn2_mape,dnn2_smape = dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape

In [107]:
lstm2_mse,lstm2_rmse,lstm2_mae,lstm2_mape,lstm2_smape = lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape

# 3

svm

In [108]:
test_X,test_Xl,test_y= test3_X,test3_Xl,test3_y

In [109]:
# svm_pred = modelsvm.predict(test_X)
# svm_pred = svm_pred[:, np.newaxis]
# svm_pred,y_true,inv_test=inv(svm_pred,test_X,test_y)

In [110]:
# svm_mse = metrics.mean_squared_error(y_true, svm_pred)
# # svm_rmse = math.sqrt(mean_squared_error(y_true[:,0], svm_pred[:,0]))
# svm_rmse = math.sqrt(mean_squared_error(y_true, svm_pred))
# svm_mae = metrics.mean_absolute_error(y_true, svm_pred)
# svm_mape = mape(y_true, svm_pred)
# svm_smape = smape(y_true, svm_pred)

In [111]:
# print(svm_mse,svm_rmse,svm_mae,svm_mape,svm_smape)

bp

In [112]:
bp_pred = modelbp.predict(test_X)
bp_pred,y_true,inv_test=inv(bp_pred,test_X,test_y)

bp_mse = metrics.mean_squared_error(y_true, bp_pred)
bp_rmse = math.sqrt(mean_squared_error(y_true[:,0], bp_pred[:,0]))
bp_mae = metrics.mean_absolute_error(y_true, bp_pred)
bp_mape = mape(y_true, bp_pred)
bp_smape = smape(y_true, bp_pred)

In [113]:
print(bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape)

0.13223447 0.3636405767459532 0.15912765 7101.8707275390625 112.58643865585327


dnn

In [114]:
dnn_pred = modeldnn.predict(test_X)
# dnn_pred,y_true,inv_test=inv(dnn_pred,test_year,test_X,test_y)
dnn_pred,y_true,inv_test=inv(dnn_pred,test_X,test_y)

dnn_mse = metrics.mean_squared_error(y_true, dnn_pred)
dnn_rmse = math.sqrt(mean_squared_error(y_true[:,0], dnn_pred[:,0]))
dnn_mae = metrics.mean_absolute_error(y_true, dnn_pred)
dnn_mape = mape(y_true, dnn_pred)
dnn_smape = smape(y_true, dnn_pred)

In [115]:
print(dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape)

0.10692455 0.32699319372909785 0.12722994 1997.8252410888672 111.62574291229248


LSTM


In [116]:
# make a prediction
lstm_pred = modellstm.predict(test_Xl)
# test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
lstm_pred,y_true,inv_test=inv(lstm_pred,test_X,test_y)
lstm_mse = metrics.mean_squared_error(y_true, lstm_pred)
lstm_rmse = math.sqrt(mean_squared_error(y_true[:,0], lstm_pred[:,0]))
lstm_mae = metrics.mean_absolute_error(y_true, lstm_pred)
lstm_mape = mape(y_true, lstm_pred)
lstm_smape = smape(y_true, lstm_pred)


In [117]:
print(lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape)

0.10337346 0.3215174341643546 0.13678373 3019.4028854370117 109.89996194839478


In [118]:
# svm3_mse,svm3_rmse,svm3_mae,svm3_mape,svm3_smape = svm_mse,svm_rmse,svm_mae,svm_mape,svm_smape

In [119]:
bp3_mse,bp3_rmse,bp3_mae,bp3_mape,bp3_smape = bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape

In [120]:
dnn3_mse,dnn3_rmse,dnn3_mae,dnn3_mape,dnn3_smape = dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape

In [121]:
lstm3_mse,lstm3_rmse,lstm3_mae,lstm3_mape,lstm3_smape = lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape

# 4

svm

In [122]:
test_X,test_Xl,test_y= test4_X,test4_Xl,test4_y

In [123]:
# svm_pred = modelsvm.predict(test_X)
# svm_pred = svm_pred[:, np.newaxis]
# svm_pred,y_true,inv_test=inv(svm_pred,test_X,test_y)

In [124]:
# svm_mse = metrics.mean_squared_error(y_true, svm_pred)
# # svm_rmse = math.sqrt(mean_squared_error(y_true[:,0], svm_pred[:,0]))
# svm_rmse = math.sqrt(mean_squared_error(y_true, svm_pred))
# svm_mae = metrics.mean_absolute_error(y_true, svm_pred)
# svm_mape = mape(y_true, svm_pred)
# svm_smape = smape(y_true, svm_pred)

In [125]:
# print(svm_mse,svm_rmse,svm_mae,svm_mape,svm_smape)

bp

In [126]:
bp_pred = modelbp.predict(test_X)
bp_pred,y_true,inv_test=inv(bp_pred,test_X,test_y)

bp_mse = metrics.mean_squared_error(y_true, bp_pred)
bp_rmse = math.sqrt(mean_squared_error(y_true[:,0], bp_pred[:,0]))
bp_mae = metrics.mean_absolute_error(y_true, bp_pred)
bp_mape = mape(y_true, bp_pred)
bp_smape = smape(y_true, bp_pred)

In [127]:
print(bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape)

0.4137377 0.6432244664894354 0.31226906 4682.763290405273 146.1649775505066


dnn

In [128]:
dnn_pred = modeldnn.predict(test_X)
# dnn_pred,y_true,inv_test=inv(dnn_pred,test_year,test_X,test_y)
dnn_pred,y_true,inv_test=inv(dnn_pred,test_X,test_y)

dnn_mse = metrics.mean_squared_error(y_true, dnn_pred)
dnn_rmse = math.sqrt(mean_squared_error(y_true[:,0], dnn_pred[:,0]))
dnn_mae = metrics.mean_absolute_error(y_true, dnn_pred)
dnn_mape = mape(y_true, dnn_pred)
dnn_smape = smape(y_true, dnn_pred)

In [129]:
print(dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape)

0.70055044 0.8369889109023917 0.38889682 4437.339782714844 148.62853288650513


LSTM


In [130]:
# make a prediction
lstm_pred = modellstm.predict(test_Xl)
# test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
lstm_pred,y_true,inv_test=inv(lstm_pred,test_X,test_y)
lstm_mse = metrics.mean_squared_error(y_true, lstm_pred)
lstm_rmse = math.sqrt(mean_squared_error(y_true[:,0], lstm_pred[:,0]))
lstm_mae = metrics.mean_absolute_error(y_true, lstm_pred)
lstm_mape = mape(y_true, lstm_pred)
lstm_smape = smape(y_true, lstm_pred)


In [131]:
print(lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape)

0.721164 0.8492137470115538 0.4026938 4355.186462402344 147.63083457946777


In [132]:
# svm4_mse,svm4_rmse,svm4_mae,svm4_mape,svm4_smape = svm_mse,svm_rmse,svm_mae,svm_mape,svm_smape

In [133]:
bp4_mse,bp4_rmse,bp4_mae,bp4_mape,bp4_smape = bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape

In [134]:
dnn4_mse,dnn4_rmse,dnn4_mae,dnn4_mape,dnn4_smape = dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape

In [135]:
lstm4_mse,lstm4_rmse,lstm4_mae,lstm4_mape,lstm4_smape = lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape

# 5

svm

In [136]:
test_X,test_Xl,test_y= test5_X,test5_Xl,test5_y

In [137]:
# svm_pred = modelsvm.predict(test_X)
# svm_pred = svm_pred[:, np.newaxis]
# svm_pred,y_true,inv_test=inv(svm_pred,test_X,test_y)

In [138]:
# svm_mse = metrics.mean_squared_error(y_true, svm_pred)
# # svm_rmse = math.sqrt(mean_squared_error(y_true[:,0], svm_pred[:,0]))
# svm_rmse = math.sqrt(mean_squared_error(y_true, svm_pred))
# svm_mae = metrics.mean_absolute_error(y_true, svm_pred)
# svm_mape = mape(y_true, svm_pred)
# svm_smape = smape(y_true, svm_pred)

In [139]:
# print(svm_mse,svm_rmse,svm_mae,svm_mape,svm_smape)

bp

In [140]:
bp_pred = modelbp.predict(test_X)
bp_pred,y_true,inv_test=inv(bp_pred,test_X,test_y)

bp_mse = metrics.mean_squared_error(y_true, bp_pred)
bp_rmse = math.sqrt(mean_squared_error(y_true[:,0], bp_pred[:,0]))
bp_mae = metrics.mean_absolute_error(y_true, bp_pred)
bp_mape = mape(y_true, bp_pred)
bp_smape = smape(y_true, bp_pred)

In [141]:
print(bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape)

0.27854386 0.5277725454400125 0.279455 470.31354904174805 108.18055868148804


dnn

In [142]:
dnn_pred = modeldnn.predict(test_X)
# dnn_pred,y_true,inv_test=inv(dnn_pred,test_year,test_X,test_y)
dnn_pred,y_true,inv_test=inv(dnn_pred,test_X,test_y)

dnn_mse = metrics.mean_squared_error(y_true, dnn_pred)
dnn_rmse = math.sqrt(mean_squared_error(y_true[:,0], dnn_pred[:,0]))
dnn_mae = metrics.mean_absolute_error(y_true, dnn_pred)
dnn_mape = mape(y_true, dnn_pred)
dnn_smape = smape(y_true, dnn_pred)

In [143]:
print(dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape)

0.30502227 0.5522882123378888 0.29530498 207.26923942565918 119.75958347320557


LSTM


In [144]:
# make a prediction
lstm_pred = modellstm.predict(test_Xl)
# test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
lstm_pred,y_true,inv_test=inv(lstm_pred,test_X,test_y)
lstm_mse = metrics.mean_squared_error(y_true, lstm_pred)
lstm_rmse = math.sqrt(mean_squared_error(y_true[:,0], lstm_pred[:,0]))
lstm_mae = metrics.mean_absolute_error(y_true, lstm_pred)
lstm_mape = mape(y_true, lstm_pred)
lstm_smape = smape(y_true, lstm_pred)


In [145]:
print(lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape)

0.3545313 0.595425300224111 0.3279283 367.0928478240967 116.59516096115112


In [146]:
# svm5_mse,svm5_rmse,svm5_mae,svm5_mape,svm5_smape = svm_mse,svm_rmse,svm_mae,svm_mape,svm_smape

In [147]:
bp5_mse,bp5_rmse,bp5_mae,bp5_mape,bp5_smape = bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape

In [148]:
dnn5_mse,dnn5_rmse,dnn5_mae,dnn5_mape,dnn5_smape = dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape

In [149]:
lstm5_mse,lstm5_rmse,lstm5_mae,lstm5_mape,lstm5_smape = lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape

# 6

svm

In [150]:
test_X,test_Xl,test_y= test6_X,test6_Xl,test6_y

In [151]:
# svm_pred = modelsvm.predict(test_X)
# svm_pred = svm_pred[:, np.newaxis]
# svm_pred,y_true,inv_test=inv(svm_pred,test_X,test_y)

In [152]:
# svm_mse = metrics.mean_squared_error(y_true, svm_pred)
# # svm_rmse = math.sqrt(mean_squared_error(y_true[:,0], svm_pred[:,0]))
# svm_rmse = math.sqrt(mean_squared_error(y_true, svm_pred))
# svm_mae = metrics.mean_absolute_error(y_true, svm_pred)
# svm_mape = mape(y_true, svm_pred)
# svm_smape = smape(y_true, svm_pred)

In [153]:
# print(svm_mse,svm_rmse,svm_mae,svm_mape,svm_smape)

bp

In [154]:
bp_pred = modelbp.predict(test_X)
bp_pred,y_true,inv_test=inv(bp_pred,test_X,test_y)

bp_mse = metrics.mean_squared_error(y_true, bp_pred)
bp_rmse = math.sqrt(mean_squared_error(y_true[:,0], bp_pred[:,0]))
bp_mae = metrics.mean_absolute_error(y_true, bp_pred)
bp_mape = mape(y_true, bp_pred)
bp_smape = smape(y_true, bp_pred)

In [155]:
print(bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape)

0.49011105 0.7000793190703177 0.3773228 166.88549518585205 113.49222660064697


dnn

In [156]:
dnn_pred = modeldnn.predict(test_X)
# dnn_pred,y_true,inv_test=inv(dnn_pred,test_year,test_X,test_y)
dnn_pred,y_true,inv_test=inv(dnn_pred,test_X,test_y)

dnn_mse = metrics.mean_squared_error(y_true, dnn_pred)
dnn_rmse = math.sqrt(mean_squared_error(y_true[:,0], dnn_pred[:,0]))
dnn_mae = metrics.mean_absolute_error(y_true, dnn_pred)
dnn_mape = mape(y_true, dnn_pred)
dnn_smape = smape(y_true, dnn_pred)

In [157]:
print(dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape)

0.61006135 0.7810642403058717 0.42621723 104.75941896438599 126.5868067741394


LSTM


In [158]:
# make a prediction
lstm_pred = modellstm.predict(test_Xl)
# test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
lstm_pred,y_true,inv_test=inv(lstm_pred,test_X,test_y)
lstm_mse = metrics.mean_squared_error(y_true, lstm_pred)
lstm_rmse = math.sqrt(mean_squared_error(y_true[:,0], lstm_pred[:,0]))
lstm_mae = metrics.mean_absolute_error(y_true, lstm_pred)
lstm_mape = mape(y_true, lstm_pred)
lstm_smape = smape(y_true, lstm_pred)


In [159]:
print(lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape)

0.6906695 0.831065266381607 0.46281114 115.80251455307007 123.5690712928772


In [160]:
# svm6_mse,svm6_rmse,svm6_mae,svm6_mape,svm6_smape = svm_mse,svm_rmse,svm_mae,svm_mape,svm_smape

In [161]:
bp6_mse,bp6_rmse,bp6_mae,bp6_mape,bp6_smape = bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape

In [162]:
dnn6_mse,dnn6_rmse,dnn6_mae,dnn6_mape,dnn6_smape = dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape

In [163]:
lstm6_mse,lstm6_rmse,lstm6_mae,lstm6_mape,lstm6_smape = lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape

# 7

svm

In [164]:
test_X,test_Xl,test_y= test7_X,test7_Xl,test7_y

In [165]:
# svm_pred = modelsvm.predict(test_X)
# svm_pred = svm_pred[:, np.newaxis]
# svm_pred,y_true,inv_test=inv(svm_pred,test_X,test_y)

In [166]:
# svm_mse = metrics.mean_squared_error(y_true, svm_pred)
# # svm_rmse = math.sqrt(mean_squared_error(y_true[:,0], svm_pred[:,0]))
# svm_rmse = math.sqrt(mean_squared_error(y_true, svm_pred))
# svm_mae = metrics.mean_absolute_error(y_true, svm_pred)
# svm_mape = mape(y_true, svm_pred)
# svm_smape = smape(y_true, svm_pred)

In [167]:
# print(svm_mse,svm_rmse,svm_mae,svm_mape,svm_smape)

bp

In [168]:
bp_pred = modelbp.predict(test_X)
bp_pred,y_true,inv_test=inv(bp_pred,test_X,test_y)

bp_mse = metrics.mean_squared_error(y_true, bp_pred)
bp_rmse = math.sqrt(mean_squared_error(y_true[:,0], bp_pred[:,0]))
bp_mae = metrics.mean_absolute_error(y_true, bp_pred)
bp_mape = mape(y_true, bp_pred)
bp_smape = smape(y_true, bp_pred)

In [169]:
print(bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape)

0.13542384 0.36799978137528105 0.16021408 564.9680614471436 77.07642316818237


dnn

In [170]:
dnn_pred = modeldnn.predict(test_X)
# dnn_pred,y_true,inv_test=inv(dnn_pred,test_year,test_X,test_y)
dnn_pred,y_true,inv_test=inv(dnn_pred,test_X,test_y)

dnn_mse = metrics.mean_squared_error(y_true, dnn_pred)
dnn_rmse = math.sqrt(mean_squared_error(y_true[:,0], dnn_pred[:,0]))
dnn_mae = metrics.mean_absolute_error(y_true, dnn_pred)
dnn_mape = mape(y_true, dnn_pred)
dnn_smape = smape(y_true, dnn_pred)

In [171]:
print(dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape)

0.1187788 0.34464300726182096 0.13558869 362.0887041091919 82.59227871894836


LSTM


In [172]:
# make a prediction
lstm_pred = modellstm.predict(test_Xl)
# test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
lstm_pred,y_true,inv_test=inv(lstm_pred,test_X,test_y)
lstm_mse = metrics.mean_squared_error(y_true, lstm_pred)
lstm_rmse = math.sqrt(mean_squared_error(y_true[:,0], lstm_pred[:,0]))
lstm_mae = metrics.mean_absolute_error(y_true, lstm_pred)
lstm_mape = mape(y_true, lstm_pred)
lstm_smape = smape(y_true, lstm_pred)


In [173]:
print(lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape)

0.13348082 0.3653502662918751 0.16057117 391.8004035949707 64.71219062805176


In [174]:
# svm7_mse,svm7_rmse,svm7_mae,svm7_mape,svm7_smape = svm_mse,svm_rmse,svm_mae,svm_mape,svm_smape

In [175]:
bp7_mse,bp7_rmse,bp7_mae,bp7_mape,bp7_smape = bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape

In [176]:
dnn7_mse,dnn7_rmse,dnn7_mae,dnn7_mape,dnn7_smape = dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape

In [177]:
lstm7_mse,lstm7_rmse,lstm7_mae,lstm7_mape,lstm7_smape = lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape

# 8

svm

In [178]:
test_X,test_Xl,test_y= test8_X,test8_Xl,test8_y

In [179]:
# svm_pred = modelsvm.predict(test_X)
# svm_pred = svm_pred[:, np.newaxis]
# svm_pred,y_true,inv_test=inv(svm_pred,test_X,test_y)

In [180]:
# svm_mse = metrics.mean_squared_error(y_true, svm_pred)
# # svm_rmse = math.sqrt(mean_squared_error(y_true[:,0], svm_pred[:,0]))
# svm_rmse = math.sqrt(mean_squared_error(y_true, svm_pred))
# svm_mae = metrics.mean_absolute_error(y_true, svm_pred)
# svm_mape = mape(y_true, svm_pred)
# svm_smape = smape(y_true, svm_pred)

In [181]:
# print(svm_mse,svm_rmse,svm_mae,svm_mape,svm_smape)

bp

In [182]:
bp_pred = modelbp.predict(test_X)
bp_pred,y_true,inv_test=inv(bp_pred,test_X,test_y)

bp_mse = metrics.mean_squared_error(y_true, bp_pred)
bp_rmse = math.sqrt(mean_squared_error(y_true[:,0], bp_pred[:,0]))
bp_mae = metrics.mean_absolute_error(y_true, bp_pred)
bp_mape = mape(y_true, bp_pred)
bp_smape = smape(y_true, bp_pred)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  


In [183]:
print(bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape)

0.11716579 0.3422948860385451 0.14862001 inf 114.03982639312744


dnn

In [184]:
dnn_pred = modeldnn.predict(test_X)
# dnn_pred,y_true,inv_test=inv(dnn_pred,test_year,test_X,test_y)
dnn_pred,y_true,inv_test=inv(dnn_pred,test_X,test_y)

dnn_mse = metrics.mean_squared_error(y_true, dnn_pred)
dnn_rmse = math.sqrt(mean_squared_error(y_true[:,0], dnn_pred[:,0]))
dnn_mae = metrics.mean_absolute_error(y_true, dnn_pred)
dnn_mape = mape(y_true, dnn_pred)
dnn_smape = smape(y_true, dnn_pred)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  


In [185]:
print(dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape)

0.103288524 0.321385319908712 0.13166212 inf 114.74963426589966


LSTM


In [186]:
# make a prediction
lstm_pred = modellstm.predict(test_Xl)
# test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
lstm_pred,y_true,inv_test=inv(lstm_pred,test_X,test_y)
lstm_mse = metrics.mean_squared_error(y_true, lstm_pred)
lstm_rmse = math.sqrt(mean_squared_error(y_true[:,0], lstm_pred[:,0]))
lstm_mae = metrics.mean_absolute_error(y_true, lstm_pred)
lstm_mape = mape(y_true, lstm_pred)
lstm_smape = smape(y_true, lstm_pred)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  


In [187]:
print(lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape)

0.111182705 0.3334407062251237 0.14885944 inf 114.88163471221924


In [188]:
# svm8_mse,svm8_rmse,svm8_mae,svm8_mape,svm8_smape = svm_mse,svm_rmse,svm_mae,svm_mape,svm_smape

In [189]:
bp8_mse,bp8_rmse,bp8_mae,bp8_mape,bp8_smape = bp_mse,bp_rmse,bp_mae,bp_mape,bp_smape

In [190]:
dnn8_mse,dnn8_rmse,dnn8_mae,dnn8_mape,dnn8_smape = dnn_mse,dnn_rmse,dnn_mae,dnn_mape,dnn_smape

In [191]:
lstm8_mse,lstm8_rmse,lstm8_mae,lstm8_mape,lstm8_smape = lstm_mse,lstm_rmse,lstm_mae,lstm_mape,lstm_smape

# 评价指标

In [195]:
#评价指标
evaluate_rmse = pd.DataFrame(columns=['评价指标','BP','DNN','LSTM'])

In [198]:
evaluate_rmse['Test']=['站点一','站点一','站点二','站点二','站点三','站点三','站点四','站点四','站点五','站点五',
                       '站点六','站点六','站点七','站点七','站点八','站点八','站点九','站点九']
evaluate_rmse=evaluate_rmse.set_index('Test',drop=True)
evaluate_rmse['评价指标']=['RMSE','MAE','RMSE','MAE','RMSE','MAE','RMSE','MAE','RMSE','MAE','RMSE','MAE','RMSE','MAE','RMSE','MAE','RMSE','MAE',]

In [199]:
evaluate_rmse

,评价指标,BP,DNN,LSTM
Test,,,,
站点一,RMSE,NaN,NaN,NaN
站点一,MAE,NaN,NaN,NaN
站点二,RMSE,NaN,NaN,NaN
站点二,MAE,NaN,NaN,NaN
站点三,RMSE,NaN,NaN,NaN
站点三,MAE,NaN,NaN,NaN
站点四,RMSE,NaN,NaN,NaN
站点四,MAE,NaN,NaN,NaN
站点五,RMSE,NaN,NaN,NaN


In [ ]:
# evaluate_rmse['SVM']=[svm0_rmse,svm1_rmse,svm2_rmse,svm3_rmse,svm4_rmse,svm5_rmse,svm6_rmse,svm7_rmse,svm8_rmse]

In [200]:
evaluate_rmse['BP']=[bp0_rmse,bp0_mae,bp1_rmse,bp1_mae,bp2_rmse,bp2_mae,bp3_rmse,bp3_mae,bp4_rmse,bp4_mae,bp5_rmse,bp5_mae,bp6_rmse,bp6_mae,bp7_rmse,bp7_mae,bp8_rmse,bp8_mae]

In [204]:
evaluate_rmse['DNN']=[dnn0_rmse,dnn0_mae,dnn1_rmse,dnn1_mae,dnn2_rmse,dnn2_mae,dnn3_rmse,dnn3_mae,dnn4_rmse,dnn4_mae,dnn5_rmse,dnn5_mae,
                      dnn6_rmse,dnn6_mae,dnn7_rmse,dnn7_mae,dnn8_rmse,dnn8_mae]

In [205]:
evaluate_rmse['LSTM']=[lstm0_rmse,lstm0_mae,lstm1_rmse,lstm1_mae,lstm2_rmse,lstm2_mae,lstm3_rmse,lstm3_mae,lstm4_rmse,lstm4_mae,lstm5_rmse,lstm5_mae,
                       lstm6_rmse,lstm6_mae,lstm7_rmse,lstm7_mae,lstm8_rmse,lstm8_mae,]

In [206]:
# s = pd.Series([evaluate_rmse['SVM'].mean(),evaluate_rmse['BP'].mean(), evaluate_rmse['DNN'].mean(), evaluate_rmse['LSTM'].mean()], index=evaluate_rmse.columns, name='mean')
# evaluate_rmse = evaluate_rmse.append(s)

In [209]:
evaluate_rmse.to_csv('../predict/九个站点评价指标.csv',index=True)
evaluate_rmse

,评价指标,BP,DNN,LSTM
Test,,,,
站点一,RMSE,0.528286,0.499608,0.545561
站点一,MAE,0.263212,0.239880,0.281347
站点二,RMSE,0.485043,0.467814,0.493364
站点二,MAE,0.261737,0.250227,0.260578
站点三,RMSE,0.342324,0.321452,0.333837
站点三,MAE,0.148159,0.140590,0.159259
站点四,RMSE,0.363641,0.326993,0.321517
站点四,MAE,0.159128,0.127230,0.136784
站点五,RMSE,0.643224,0.836989,0.849214


# MAE

In [ ]:
#评价指标
evaluate_mae = pd.DataFrame(columns=['Test','SVM','BP','DNN','LSTM'])

In [ ]:
evaluate_mae['Test']=['0','1','2','3','4','5','6','7','8']
evaluate_mae=evaluate_mae.set_index('Test',drop=True)

In [ ]:
evaluate_mae

In [ ]:
# evaluate_mae['SVM']=[svm0_mae,svm1_mae,svm2_mae,svm3_mae,svm4_mae,svm5_mae,svm6_mae,svm7_mae,svm8_mae]

In [ ]:
evaluate_mae['BP']=[bp0_mae,bp1_mae,bp2_mae,bp3_mae,bp4_mae,bp5_mae,bp6_mae,bp7_mae,bp8_mae]

In [ ]:
evaluate_mae['DNN']=[dnn0_mae,dnn1_mae,dnn2_mae,dnn3_mae,dnn4_mae,dnn5_mae,dnn6_mae,dnn7_mae,dnn8_mae]

In [ ]:
evaluate_mae['LSTM']=[lstm0_mae,lstm1_mae,lstm2_mae,lstm3_mae,lstm4_mae,lstm5_mae,lstm6_mae,lstm7_mae,lstm8_mae]

In [ ]:
s = pd.Series([evaluate_mae['SVM'].mean(),evaluate_mae['BP'].mean(), evaluate_mae['DNN'].mean(), evaluate_mae['LSTM'].mean()], index=evaluate_mae.columns, name='mean')
evaluate_mae = evaluate_mae.append(s)

In [ ]:
# evaluate_mae.to_csv('../predict/maetest0-8.csv',index=True)
evaluate_mae

# 保存预测结果并绘图

In [ ]:
#真实值和预测值对比图
plt.plot(y_true,'royalblue',label='Real')
plt.plot(svm_pred,'orange',label='SVM')
# plt.plot(bp_pred,label='BP')
# plt.plot(dnn_pred,label='DNN')
# plt.plot(lstm_pred,label='LSTM')
plt.title('1A Trina Prediction')
plt.ylabel('Power')
plt.xlabel('Time')
plt.legend()
plt.savefig('../picture/SVM.jpg')

In [ ]:
#真实值和预测值对比图
plt.plot(y_true,'royalblue',label='Real')
# plt.plot(svm_pred,label='SVM')
plt.plot(bp_pred,'limegreen',label='BP')
# plt.plot(dnn_pred,label='DNN')
# plt.plot(lstm_pred,label='LSTM')

plt.title('1A Trina Prediction')
plt.ylabel('Power')
plt.xlabel('Time')
plt.legend()
plt.savefig('../picture/BP.jpg')

In [ ]:
#真实值和预测值对比图
plt.plot(y_true,'royalblue',label='Real')
# plt.plot(svm_pred,label='SVM')
# plt.plot(bp_pred,label='BP')
plt.plot(dnn_pred,'tomato',label='DNN')
# plt.plot(lstm_pred,label='LSTM')

plt.title('1A Trina Prediction')
plt.ylabel('Power')
plt.xlabel('Time')
plt.legend()
plt.savefig('../picture/DNN.jpg')

In [ ]:
#真实值和预测值对比图
plt.plot(y_true,'royalblue',label='Real')
# plt.plot(svm_pred,label='SVM')
# plt.plot(bp_pred,label='BP')
# plt.plot(dnn_pred,label='DNN')
plt.plot(lstm_pred,'orchid',label='LSTM')

plt.title('1A Trina Prediction')
plt.ylabel('Power')
plt.xlabel('Time')
plt.legend()
plt.savefig('../picture/LSTM.jpg')

# 保存

In [ ]:
#真实值和预测值对比图
plt.plot(y_true,label='Real')
plt.plot(svm_pred,label='SVM')
plt.plot(bp_pred,label='BP')
plt.plot(dnn_pred,label='DNN')
plt.plot(lstm_pred,label='LSTM')

plt.title('1A Trina Prediction')
plt.ylabel('Power')
plt.xlabel('Time')
plt.legend()
plt.savefig('../picture/SVMLSTM.jpg')

#训练集和测试集 预测结果保存
predict_ = pd.DataFrame(inv_test)
# predict_.columns = ['Year', 'Month', 'Day', 'Hour', 'Minutes', 'WTC', 'WRH', 'GHR', 'DHR',
#        'RGT', 'RDT', '1A Trina_power']
predict_.columns = ['Month(t-1)', 'Day(t-1)', 'Hour(t-1)', 'Minutes(t-1)', 'WTC(t-1)',
       'WRH(t-1)', 'GHR(t-1)', 'DHR(t-1)', 'WD(t-1)', 'WDR(t-1)', 'RGT(t-1)',
       'RDT(t-1)', '1A Trina_power(t-1)', '1A Trina_power(t)',
       '1A Trina_power(t+1)']
predict_['SVM'] = svm_pred
predict_['BP'] = bp_pred
predict_['DNN'] = dnn_pred
predict_['LSTM'] = lstm_pred
predict_.to_csv('../predict/1Atest_pred_-WDWDR_svm&bp&dnn.csv',index=False)


画出例如2020年1月20号的预测图

In [ ]:
predict_.columns

In [ ]:
# from scipy.interpolate import spline
predict_[['Month(t-1)', 'Day(t-1)', 'Hour(t-1)', 'Minutes(t-1)']] = predict_[['Month(t-1)', 'Day(t-1)', 'Hour(t-1)', 'Minutes(t-1)']].astype("int")

# Day0120 = predict_.loc[(predict_['Year'] == 2020) & (predict_['Month'] == ) & (predict_['Day'] == 20),:]
Day0120 = predict_.loc[(predict_['Month(t-1)'] == 2 ) & (predict_['Day(t-1)'] == 20),:]
#real_power_day.set_index('time', inplace=False)
Real=Day0120.set_index('Hour(t-1)', inplace=False)['1A Trina_power(t+1)']
svm_pr=Day0120.set_index('Hour(t-1)', inplace=False)['SVM']
bp_pr=Day0120.set_index('Hour(t-1)', inplace=False)['BP']
dnn_pr=Day0120.set_index('Hour(t-1)', inplace=False)['DNN']
lstm_pr=Day0120.set_index('Hour(t-1)', inplace=False)['LSTM']

fig = plt.figure()
plt.plot(Real,label='Real')
plt.plot(svm_pr,label='SVM')
plt.plot(bp_pr,label='BP')
plt.plot(dnn_pr,label='DNN')
plt.plot(lstm_pr,label='LSTM')

plt.title('Prediction 2020-2-20')
plt.ylabel('Power')
plt.xlabel('Time')
# plt.legend(['Train', 'Test'], loc='upper left')
plt.legend()
plt.savefig('../picture/2-20pred.jpg')
#ax = fig.add_subplot(1,1,1)
#ax.plot(real_power_day_)
# real_power_day_.plot()
#real_power_day_.set_axis(['00:15:00','06:15:00','12:15:00','18:15:00'])
#real_power_day_

# 2020 

In [ ]:
# from scipy.interpolate import spline
predict_[['Month(t-1)', 'Day(t-1)', 'Hour(t-1)', 'Minutes(t-1)']] = predict_[['Month(t-1)', 'Day(t-1)', 'Hour(t-1)', 'Minutes(t-1)']].astype("int")

# Day0120 = predict_.loc[(predict_['Year'] == 2020) & (predict_['Month'] == ) & (predict_['Day'] == 20),:]
Day0120 = predict_.loc[(predict_['Month(t-1)'] == 3 ) & (predict_['Day(t-1)'] == 25),:]
#real_power_day.set_index('time', inplace=False)
Real=Day0120.set_index('Hour(t-1)', inplace=False)['1A Trina_power(t+1)']
svm_pr=Day0120.set_index('Hour(t-1)', inplace=False)['SVM']
bp_pr=Day0120.set_index('Hour(t-1)', inplace=False)['BP']
dnn_pr=Day0120.set_index('Hour(t-1)', inplace=False)['DNN']
lstm_pr=Day0120.set_index('Hour(t-1)', inplace=False)['LSTM']

fig = plt.figure()
plt.plot(Real,label='Real')
plt.plot(svm_pr,label='SVM')
plt.plot(bp_pr,label='BP')
plt.plot(dnn_pr,label='DNN')
plt.plot(lstm_pr,label='LSTM')

plt.title('Prediction 2020-3-25')
plt.ylabel('Power')
plt.xlabel('Time')
# plt.legend(['Train', 'Test'], loc='upper left')
plt.legend()
plt.savefig('../picture/3-25pred.jpg')
#ax = fig.add_subplot(1,1,1)
#ax.plot(real_power_day_)
# real_power_day_.plot()
#real_power_day_.set_axis(['00:15:00','06:15:00','12:15:00','18:15:00'])
#real_power_day_